Prerequisites:

- Target workspace and Lakehouse must exist
- Tested on PySpark and Python kernels
- Note: target lakehouse must exist!

In [ ]:
%pip install semantic-link-labs

In [ ]:
# Original Author Nalaka, modified Brett Cooper
# modified from https://bidiaries.com/simplify-shortcuts-bulk-creation-with-pyspark-semantic-link-labs-in-microsoft-fabric#heading-scenario-2-bulk-shortcut-creation-from-existing-source-tables

import sempy_labs
from datetime import datetime

# Parameters for source and destination lakehouse and Workspaces
source_lakehouse = "SOURCE_LAKEHOUSE_NAME"
source_workspace = "SOURCE_WORKSPACE_NAME"
destination_lakehouse = "TARGET_LAKEHOSUE_NAME" # note: target lakehouse must exist!
destination_workspace = "TARGET_WORKSPACE_NAME"

In [ ]:
# Fetch the list of tables from the source lakehouse
# reminder that the target lakehouse needs to exist
# This is done once and reused to improve perf, do not rerun cells below without refreshing this list
tables = sempy_labs.lakehouse.get_lakehouse_tables(
    lakehouse=source_lakehouse,
    workspace=source_workspace
)

# to filter just based on a specific schema
# tables = tables[tables['Schema Name'] == 'finance']

tables_in_destination = sempy_labs.lakehouse.get_lakehouse_tables(
    lakehouse=destination_lakehouse,
    workspace=destination_workspace
)

In [ ]:
# check the content to be worked on
# tables

In [ ]:
# Function to check for existing shortcuts
def shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name, schema_name):
    exists = ((tables_in_destination['Schema Name'] == schema_name) & 
            (tables_in_destination['Table Name'] == shortcut_name)).any()
            
    return exists

In [ ]:
#what to do if the target shortcut already exists
# 'Replace' or 'Skip' 
shortcut_duplicate_behaviour = 'Skip'  # Replace - tries to delete and recreate the sc, or 'Skip' - or any_other_string this gives the same behaviour as Abort as in the documentation from sem link labs

In [ ]:
# Iterate through each row in the DataFrame to process the 'Table Name'
for _, row in tables.iterrows():
    shortcut_name = row['Table Name']  # Extract 'Table Name' column as shortcut name
    schema_name = row['Schema Name']
    source_path = f"Tables/{schema_name}"
    destination_path = f"Tables/{schema_name}"
    
    # Skip processing for shortcut_name 'schema.json.gz'
    if shortcut_name == 'schema.json.gz':
        continue
    
    print(f"\n ∟ {datetime.now().strftime('%H:%M:%S')} Processing shortcut: {shortcut_name} in schema {schema_name}")


    # Check if the shortcut already exists
    if shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name, schema_name) and shortcut_duplicate_behaviour == 'Replace':
            try:
                # Delete the existing shortcut
                sempy_labs.lakehouse.delete_shortcut(
                        shortcut_name=shortcut_name,
                        lakehouse=destination_lakehouse,
                        workspace=destination_workspace,
                        shortcut_path=destination_path
                    )
            except Exception as e:
                print(f"🔴 Failed to delete existing shortcut '{shortcut_name}'. Error: {e}")
                continue  # Skip to the next shortcut if delete fails
    elif shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name, schema_name):
        print(f"🔵 The shortcut named {shortcut_name} in schema {schema_name} in lakehouse {destination_workspace} has been skipped, already exists.")
        continue
    # Create a new shortcut
    
    try:
        sempy_labs.lakehouse.create_shortcut_onelake(
            table_name=shortcut_name, source_lakehouse=source_lakehouse, source_workspace=source_workspace, source_path=source_path, 
            destination_lakehouse=destination_lakehouse,destination_workspace=destination_workspace, destination_path=destination_path
            )
    except Exception as e:
        print(f"🔴 Failed to create shortcut for '{shortcut_name}'. Error: {e}")

print("\n\n ∟ 🟢🟢🟢 All shortcuts processed successfully.")

### Delete the target lakehouse shortcuts - for testing

In [ ]:
# refresh the tables list
tables_in_destination = sempy_labs.lakehouse.get_lakehouse_tables(
    lakehouse=destination_lakehouse,
    workspace=destination_workspace
)
print(f"\n ∟ {datetime.now().strftime('%H:%M:%S')} Deleteing shortcuts")
# Iterate through each row in the DataFrame to process the 'Table Name'
for _, row in tables_in_destination.iterrows():
    shortcut_name = row['Table Name']  # Extract 'Table Name' column as shortcut name
    schema_name = row['Schema Name']
    destination_path = f"Tables/{schema_name}"       
    # Delete the existing shortcut
    sempy_labs.lakehouse.delete_shortcut(
            shortcut_name=shortcut_name,
            lakehouse=destination_lakehouse,
            workspace=destination_workspace,
            shortcut_path=destination_path)
